In [1]:
import sys
from pathlib import Path

In [2]:
import pandas as pd
import altair as alt

In [3]:
this_dir = Path("__file__").parent.absolute()

In [4]:
sys.path.append(str(this_dir.parent / "newshomepages"))

In [5]:
import altair_theme

In [6]:
alt.themes.register('palewire', altair_theme.theme)
alt.themes.enable('palewire')

ThemeRegistry.enable('palewire')

In [7]:
sources_dir = this_dir.parent / "newshomepages" / "sources"

In [8]:
extracts_dir = this_dir.parent / "extracts" / "csv"

In [9]:
screenshots_df = pd.read_csv(
    extracts_dir / "screenshot-files.csv",
    parse_dates=["mtime"],
    usecols=["identifier", "handle", "file_name", "mtime"]
)

In [10]:
screenshots_df['date'] = pd.to_datetime(screenshots_df.mtime.dt.date)

In [11]:
site_df = screenshots_df[screenshots_df.handle == 'latimes']

In [12]:
site_by_date = site_df.groupby("date").size().rename("screenshots").reset_index()

In [13]:
site_by_date['rolling_mean'] = site_by_date.screenshots.rolling(7).mean()

In [14]:
site_by_date_nonulls = site_by_date[~pd.isnull(site_by_date.rolling_mean)]

In [17]:
chart = alt.Chart(
    "https://raw.githubusercontent.com/palewire/news-homepages/main/docs/_extra/charts/sites/screenshots/latimes.json",
    title="Screenshots saved by day",
)

line = chart.mark_area(line=True).encode(
    x=alt.X("date:T", title=None, timeUnit="yearmonthdate", axis=alt.Axis(format="%B %-d", grid=False)),
    y=alt.Y("value:Q", title="Seven-day rolling average"),
)

(line).configure_axisY(
    labelPadding=5, 
    labelAlign='right',
    labelFlush=True,
)# .properties(width=500, height=300)

alt.Chart(...)